Cell 1 → Install Dependencies

In [ ]:
!pip install streamlit pandas numpy sentence-transformers torch

Cell 2 → Imports + Utility Functions (utils.py content)

In [ ]:
import pandas as pd
import numpy as np
import re

def preprocess_rubric(df):
    df = df.copy()
    df.columns = [str(c).strip().lower() for c in df.columns]
    df["keyword_list"] = df["keywords"].fillna("").apply(
        lambda x: [k.strip().lower() for k in str(x).split(",") if k.strip()]
    )
    df["weight"] = pd.to_numeric(df.get("weight", 1.0), errors="coerce").fillna(1.0)
    df["min_words"] = pd.to_numeric(df.get("min_words", np.nan), errors="coerce")
    df["max_words"] = pd.to_numeric(df.get("max_words", np.nan), errors="coerce")
    return df

def normalize_text(text: str) -> str:
    return re.sub(r"\s+", " ", text.lower()).strip()

Cell 3 → Scoring Logic (scoring.py content)

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def compute_scores_for_transcript(transcript: str, rubric_df):
    transcript_clean = normalize_text(transcript)
    words = transcript_clean.split()
    word_count = len(words)

    transcript_emb = model.encode(transcript_clean, convert_to_tensor=True)

    results = []

    for _, row in rubric_df.iterrows():
        criterion = row["criterion"]
        description = str(row["description"])
        keywords = row["keyword_list"]
        weight = row["weight"]
        min_w = row.get("min_words", np.nan)
        max_w = row.get("max_words", np.nan)

        # Keyword score
        found_keywords = [k for k in keywords if k in transcript_clean]
        keyword_score = (len(found_keywords) / len(keywords)) if len(keywords) > 0 else 1.0

        # Length score
        if not np.isnan(min_w) and word_count < min_w:
            length_score = word_count / min_w
        elif not np.isnan(max_w) and word_count > max_w:
            length_score = max_w / word_count
        else:
            length_score = 1.0

        # Semantic score
        desc_emb = model.encode(description, convert_to_tensor=True)
        sim = util.cos_sim(transcript_emb, desc_emb).item()
        semantic_score = (sim + 1) / 2

        combined = (
            0.4 * keyword_score +
            0.4 * semantic_score +
            0.2 * length_score
        )

        results.append({
            "criterion": criterion,
            "keyword_score": round(keyword_score, 3),
            "semantic_score": round(semantic_score, 3),
            "length_score": round(length_score, 3),
            "combined_score": combined,
            "found_keywords": found_keywords,
            "weight": weight,
            "similarity_raw": round(sim, 3),
            "word_count": word_count
        })

    total_weight = sum(r["weight"] for r in results)
    weighted_sum = sum(r["combined_score"] * r["weight"] for r in results)
    overall_100 = (weighted_sum / total_weight) * 100

    return round(overall_100, 1), results

Cell 4 → Load Rubric

In [ ]:
# CODE 4: Load Rubric Excel File

import pandas as pd

# Path to the file you uploaded here
RUBRIC_PATH = "/content/Case study for interns.xlsx"

# IMPORTANT: Check your sheet name!
# If your rubric sheet is not named "Rubric",
# run: pd.ExcelFile(RUBRIC_PATH).sheet_names
# and use the correct sheet.
RUBRIC_SHEET = "Rubrics"

# Load rubric, assuming the actual header is in the 13th row (index 12).
# This will make the first row of the DataFrame the one with 'Creteria', 'Metric', 'Weightage'.
temp_rubric_df = pd.read_excel(RUBRIC_PATH, sheet_name=RUBRIC_SHEET, header=12)

# The actual data starts from the second row (index 1) after loading with header=12.
# Let's drop the first row as it contains header descriptions, not actual criteria data.
rubric_df = temp_rubric_df.iloc[1:].copy()

# Rename columns to match the expected format for the scoring logic.
# We'll create 'criterion' and 'description' from 'Unnamed: 2'
rubric_df['criterion'] = rubric_df['Unnamed: 2']
rubric_df['description'] = rubric_df['Unnamed: 2'] # For now, description is the same as criterion

rubric_df = rubric_df.rename(columns={
    'Unnamed: 3': 'weight',
    'Unnamed: 4': 'max_words' # Assuming Unnamed: 4 holds max_words or similar
})

# Select and reorder the essential columns
rubric_df = rubric_df[['criterion', 'description', 'weight', 'max_words']]

# Fill NaN in 'criterion' and 'description' with empty strings to avoid errors later
rubric_df['criterion'] = rubric_df['criterion'].fillna('')
rubric_df['description'] = rubric_df['description'].fillna('')

# --- NEW: Filter out empty or irrelevant criteria rows ---
# Remove rows where 'criterion' is empty or just 'Metric' as these are not actual scoring criteria
rubric_df = rubric_df[rubric_df['criterion'].str.strip() != ''].copy()
rubric_df = rubric_df[rubric_df['criterion'].str.strip().str.lower() != 'metric'].copy() # Fix: use .str.lower()

print("Rubric loaded and partially processed!")
print("Columns found:", rubric_df.columns.tolist())

rubric_df.head()

Rubric loaded and partially processed!
Columns found: ['criterion', 'description', 'weight', 'max_words']


,criterion,description,weight,max_words
1,Salutation Level,Salutation Level,5,40
2,"Key word Presence (Does this include name, age...","Key word Presence (Does this include name, age...",30,NaN
3,"Flow (Is the introduction following the order,...","Flow (Is the introduction following the order,...",5,NaN
4,Speech rate(words/minute),Speech rate(words/minute),10,10
5,Grammar errors count using Language Tool,Grammar errors count using Language Tool,10,20


Code 5: Preprocess the Rubric

In [ ]:
# CODE 5: Preprocess Rubric

import numpy as np
import re
import pandas as pd # Ensure pandas is imported if it's not already in the cell

def preprocess_rubric(df):
    df = df.copy()

    # clean column names
    df.columns = [c.strip().lower() for c in df.columns]

    # ensure keywords column exists
    # The previous cell does not create a 'keywords' column, so it will be missing
    # Let's assume 'description' can act as 'keywords' for now if no explicit 'keywords' column is present.
    # Or, if keywords are meant to be parsed from 'description', that logic should be here.
    # For this specific file, it seems the rubric does not have an explicit 'keywords' column,
    # so we'll ensure 'keywords' is an empty string column to prevent errors.
    if "keywords" not in df.columns:
        df["keywords"] = ""

    # convert keywords into list
    df["keyword_list"] = df["keywords"].fillna("").apply(
        lambda x: [k.strip().lower() for k in str(x).split(",") if k.strip()]
    )

    # normalize numeric weights
    if "weight" not in df.columns:
        df["weight"] = 1.0 # Create the column with default value 1.0
    df["weight"] = pd.to_numeric(df["weight"], errors="coerce").fillna(1.0)

    # min/max words
    # It seems the rubric has 'max_words' but not 'min_words' explicitly.
    # df.get will handle missing 'min_words' by providing np.nan as default.
    df["min_words"] = pd.to_numeric(df.get("min_words", np.nan), errors="coerce")
    df["max_words"] = pd.to_numeric(df.get("max_words", np.nan), errors="coerce")

    return df

# apply preprocessing
rubric_df = preprocess_rubric(rubric_df)

print("Rubric preprocessed!")
rubric_df.head()

Rubric preprocessed!


,criterion,description,weight,max_words,keywords,keyword_list,min_words
1,Salutation Level,Salutation Level,5.0,40.0,,[],NaN
2,"Key word Presence (Does this include name, age...","Key word Presence (Does this include name, age...",30.0,NaN,,[],NaN
3,"Flow (Is the introduction following the order,...","Flow (Is the introduction following the order,...",5.0,NaN,,[],NaN
4,Speech rate(words/minute),Speech rate(words/minute),10.0,10.0,,[],NaN
5,Grammar errors count using Language Tool,Grammar errors count using Language Tool,10.0,20.0,,[],NaN


Code 6 — Load SentenceTransformer Model & Precompute Rubric Embeddings

In [ ]:
# CODE 6: Load Sentence Transformer model and precompute rubric embeddings

!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer, util
import numpy as np

# Load a lightweight, good general-purpose model
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = SentenceTransformer(MODEL_NAME)

print("Model loaded:", MODEL_NAME)

# Make sure rubric_df exists from previous cells (Code 4 & 5)
# We'll embed the 'description' of each criterion once and reuse it.
if "description" not in rubric_df.columns:
    raise ValueError("rubric_df must have a 'description' column before running Code 6")

rubric_texts = rubric_df["description"].fillna("").astype(str).tolist()
rubric_embeddings = embed_model.encode(rubric_texts, convert_to_tensor=True, show_progress_bar=True)

# Store embeddings in the DataFrame (as a list of vectors)
rubric_df["desc_embedding"] = list(rubric_embeddings.cpu().numpy())

print("Rubric embeddings computed and stored in rubric_df['desc_embedding']!")
rubric_df.head()

Model loaded: sentence-transformers/all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Rubric embeddings computed and stored in rubric_df['desc_embedding']!


,criterion,description,weight,max_words,keywords,keyword_list,min_words,desc_embedding
1,Salutation Level,Salutation Level,5.0,40.0,,[],NaN,"[-0.026852787, 0.049311385, 0.045291185, 0.028..."
2,"Key word Presence (Does this include name, age...","Key word Presence (Does this include name, age...",30.0,NaN,,[],NaN,"[0.05892347, -0.037185486, 0.0065952283, 0.025..."
3,"Flow (Is the introduction following the order,...","Flow (Is the introduction following the order,...",5.0,NaN,,[],NaN,"[-0.04237169, 0.10924154, 0.033409987, 0.01384..."
4,Speech rate(words/minute),Speech rate(words/minute),10.0,10.0,,[],NaN,"[0.04669899, -0.019006375, -0.045036774, -0.06..."
5,Grammar errors count using Language Tool,Grammar errors count using Language Tool,10.0,20.0,,[],NaN,"[0.038657233, -0.026178386, -0.01735092, 0.061..."


In [ ]:
# QUICK TEST

sample_text = """
Hi, my name is Chandan. I am from Bangalore and currently pursuing my engineering.
I enjoy working in teams and have experience in public speaking and debate clubs.
"""

result = score_transcript(sample_text, rubric_df, embed_model)
result["overall_score"], result["words"]

(53.56, 28)

Code 7 — Scoring Functions (Rule-based + Semantic + Length)

In [ ]:
# CODE 7: Define scoring functions and main scoring pipeline

import numpy as np
import re
import torch # Import torch for tensor operations
from sentence_transformers import SentenceTransformer, util

def clean_text(text: str) -> str:
    """Basic cleaning: lowercase and strip extra spaces."""
    return re.sub(r"\s+", " ", str(text).strip().lower())

def tokenize_words(text: str):
    """Simple word tokenization by splitting on whitespace."""
    return [w for w in clean_text(text).split(" ") if w]

def compute_keyword_score(transcript_tokens, keyword_list):
    """
    Compute ratio of keywords present in the transcript.
    Returns a score in [0, 1] or np.nan if no keywords are defined.
    """
    if not keyword_list:
        return np.nan  # no keyword constraint for this criterion

    transcript_set = set(transcript_tokens)
    found = sum(1 for kw in keyword_list if kw in transcript_set)
    total = len(keyword_list)
    return found / total if total > 0 else np.nan

def compute_length_score(num_words, min_words, max_words):
    """
    Length score in [0, 1]:
    - 1.0 if within [min_words, max_words]
    - decays linearly if outside the range
    - np.nan if both min and max are missing
    """
    if np.isnan(min_words) and np.isnan(max_words):
        return np.nan

    # Handle missing min/max gracefully
    if np.isnan(min_words):
        min_words = 0
    if np.isnan(max_words):
        max_words = num_words  # treat as no upper bound

    if min_words <= num_words <= max_words:
        return 1.0
    elif num_words < min_words:
        # linearly down to 0 when num_words = 0
        return max(num_words / max(min_words, 1), 0.0)
    else:
        # num_words > max_words: decay as we exceed max_words
        # after 2x max_words, score ~0
        return max(1.0 - (num_words - max_words) / max(max_words, 1), 0.0)

def compute_semantic_score(transcript_embedding, criterion_embedding):
    """
    Compute cosine similarity in [-1, 1], then scale to [0, 1].
    """
    sim = util.cos_sim(transcript_embedding, criterion_embedding)[0][0].item()
    # Map from [-1,1] to [0,1]
    return (sim + 1.0) / 2.0

def score_transcript(transcript_text: str, rubric_df, model):
    """
    Main scoring function.
    Returns:
      - overall_score (0–100)
      - details: list of dicts, one per criterion
    """
    # Clean and tokenize transcript
    transcript_text_clean = clean_text(transcript_text)
    transcript_tokens = tokenize_words(transcript_text_clean)
    num_words = len(transcript_tokens)

    # Embed transcript once
    transcript_embedding = model.encode([transcript_text_clean], convert_to_tensor=True)

    criterion_details = []
    weighted_scores = []
    weight_sum = 0.0

    for idx, row in rubric_df.iterrows():
        criterion = str(row.get("criterion", f"Criterion {idx+1}"))
        description = str(row.get("description", ""))
        weight = float(row.get("weight", 1.0))

        keyword_list = row.get("keyword_list", [])
        if not isinstance(keyword_list, list):
            keyword_list = []

        min_words = row.get("min_words", np.nan)
        max_words = row.get("max_words", np.nan)

        # Embedding for this criterion
        criterion_emb_vec = row.get("desc_embedding", None)
        if criterion_emb_vec is None:
            # In case something went wrong, compute on the fly
            criterion_embedding = model.encode([description], convert_to_tensor=True)
        else:
            # Fix: Use torch.tensor().to() for device placement
            criterion_embedding = torch.tensor(criterion_emb_vec).to(transcript_embedding.device).unsqueeze(0)

        # --- Component scores ---
        keyword_score = compute_keyword_score(transcript_tokens, keyword_list)
        length_score = compute_length_score(num_words, min_words, max_words)
        semantic_score = compute_semantic_score(transcript_embedding, criterion_embedding)

        # Decide dynamic weights for combination
        # Start with a strong emphasis on semantic similarity
        semantic_weight = 0.7
        keyword_weight = 0.2 if not np.isnan(keyword_score) else 0.0
        length_weight = 0.1 if not np.isnan(length_score) else 0.0

        total_component_weight = semantic_weight + keyword_weight + length_weight
        if total_component_weight == 0:
            total_component_weight = 1.0  # avoid division by zero

        # Combine into final per-criterion score in [0,1]
        combined_score = (
            semantic_weight * semantic_score +
            keyword_weight * (0.0 if np.isnan(keyword_score) else keyword_score) +
            length_weight * (0.0 if np.isnan(length_score) else length_score)
        ) / total_component_weight

        # Convert to 0–100
        criterion_score_100 = combined_score * 100.0

        # Aggregate for overall score
        weighted_scores.append(criterion_score_100 * weight)
        weight_sum += weight

        # Build feedback text
        fb_parts = [
            f"Semantic alignment: {semantic_score:.2f}",
        ]
        if not np.isnan(keyword_score):
            fb_parts.append(f"Keyword coverage: {keyword_score:.2f}")
        if not np.isnan(length_score):
            fb_parts.append(f"Length score: {length_score:.2f}")
        fb_parts.append(f"Words in transcript: {num_words}")

        feedback = "; ".join(fb_parts)

        criterion_details.append({
            "criterion": criterion,
            "description": description,
            "weight": weight,
            "score": round(criterion_score_100, 2),
            "semantic_score": round(semantic_score, 3),
            "keyword_score": None if np.isnan(keyword_score) else round(keyword_score, 3),
            "length_score": None if np.isnan(length_score) else round(length_score, 3),
            "words": num_words,
            "feedback": feedback,
        })

    # Overall weighted score
    if weight_sum == 0:
        overall_score = 0.0
    else:
        overall_score = sum(weighted_scores) / weight_sum

    overall_score = round(overall_score, 2)

    result = {
        "overall_score": overall_score,
        "details": criterion_details,
        "words": num_words,
    }
    return result

print("Scoring functions defined! Use score_transcript(transcript_text, rubric_df, embed_model).")

Scoring functions defined! Use score_transcript(transcript_text, rubric_df, embed_model).


In [ ]:
# QUICK TEST

sample_text = """
Hi, my name is Chandan. I am from Bangalore and currently pursuing my engineering.
I enjoy working in teams and have experience in public speaking and debate clubs.
"""

result = score_transcript(sample_text, rubric_df, embed_model)
result["overall_score"], result["words"]


(53.56, 28)

Code 8 — Simple Gradio Frontend (Paste this next)

In [ ]:
# CODE 8: Simple Gradio Web App
!pip install -q gradio

import gradio as gr
import json

def evaluate(transcript):
    if not transcript.strip():
        return 0.0, 0, "Please enter a transcript."

    result = score_transcript(transcript, rubric_df, embed_model)

    overall = result["overall_score"]
    words = result["words"]
    pretty_json = json.dumps(result, indent=2)

    return overall, words, pretty_json


with gr.Blocks() as demo:
    gr.Markdown("## 🧠 AI Communication Scoring Tool\nPaste transcript & get your score instantly.")

    transcript_box = gr.Textbox(
        label="Transcript",
        placeholder="Paste self-introduction transcript here...",
        lines=8
    )

    overall_out = gr.Number(label="Overall Score (0–100)")
    word_out = gr.Number(label="Word Count")
    json_out = gr.Textbox(label="Detailed Output (JSON)", lines=20)

    btn = gr.Button("Score")

    btn.click(
        evaluate,
        inputs=[transcript_box],
        outputs=[overall_out, word_out, json_out]
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://63acccb08c73ed00de.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
